## Setup

In [ ]:
# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyjwt

## Dependencies

In [ ]:
import requests
import json
import jwt
import time

## Global variables

In [ ]:
BASE_URL = 'https://gateway.dev.wadmp.com'
BASE_PATH = 'api'

## User input

In [ ]:
USERNAME = input("Enter username:")
PASSWORD = input("Enter password:")

## Login

In [ ]:
url = f"{BASE_URL}/public/auth/connect/token"
credentials = {'username': USERNAME, 'password': PASSWORD, 'client_id': 'python', 'grant_type': 'password'}
print(f"\nSending POST request to {url} with:\n"
        f"    credentials={credentials}")
response = requests.post(url, data=credentials)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    USER_TOKEN = response.json()["access_token"]
else:
    print("Failed to login!")
    sys.exit(1)

## Analyse the JSON Web Token (JWT)

In [ ]:
headers = jwt.get_unverified_header(USER_TOKEN)
print(json.dumps(headers, indent=4, sort_keys=True))

In [ ]:
claims = jwt.decode(USER_TOKEN, verify=False)
print(json.dumps(claims, indent=4, sort_keys=True))

In [ ]:
nbf = claims['nbf']
print(f"Token valid from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(nbf))} (local time)")
seconds = int(time.time()) - nbf
print(f"i.e. {seconds} seconds ago")

In [ ]:
exp = claims['exp']
print(f"Token expires {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(exp))} (local time)")
seconds = exp - int(time.time())
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print(f"i.e. In {seconds} seconds, or {h:d}h {m:02d}m {s:02d}s")